### Before Start: Plotting Function: show()

In [1]:
#### The Plotting Functions ####
import matplotlib.pyplot as plt
import numpy as np
def show(ori_func, ft, sampling_period = 5):
    n = len(ori_func)
    interval = sampling_period / n
    plt.subplot(2, 1, 1)
    plt.plot(np.arange(0, sampling_period, interval), ori_func, 'black')
    plt.xlabel('Time'), plt.ylabel('Amplitude')
    plt.subplot(2,1,2)
    frequency = np.arange(n / 2) / (n * interval)
    nfft = abs(ft[range(int(n / 2))] / n )
    plt.plot(frequency, nfft, 'red')
    plt.xlabel('Freq (Hz)'), plt.ylabel('Amp. Spectrum')
    plt.show()  

### Signal Processing

In [2]:
#### Sine Signal ####
time = np.arange(0, 5, .005)
x = np.sin(2 * np.pi * 1 * time)
y = np.fft.fft(x)
show(x, y)  

In [3]:
#### Sine Signal with Different Frequency ####
x2 = np.sin(2 * np.pi * 20 * time)
x3 = np.sin(2 * np.pi * 60 * time)
x += x2 + x3
y = np.fft.fft(x)
show(x, y)

In [4]:
#### Square Signal ####
x = np.zeros(len(time))
x[::20] = 1
y = np.fft.fft(x)
show(x, y)

In [5]:
#### Pulse Signal ####
x = np.zeros(len(time))
x[380:400] = np.arange(0, 1, .05)
x[400:420] = np.arange(1, 0, -.05)
y = np.fft.fft(x)
show(x, y)

In [6]:
#### Random Signal ####
x = np.random.random(100)
y = np.fft.fft(x)
show(x, y)

### Fourier Transformation

In [7]:
#### Fourier Equation ####
x = np.random.random(500)
n = len(x)
m = np.arange(n)
k = m.reshape((n, 1))
M = np.exp(-2j * np.pi * k * m / n)
y = np.dot(M, x)
np.allclose(y, np.fft.fft(x))

True

In [8]:
#### Compare Performance between Manual Calculation and np.fft ####
%timeit np.dot(np.exp(-2j * np.pi * np.arange(n).reshape((n, 1)) * np.arange(n) / n), x)
%timeit np.fft.fft(x)

10 loops, best of 3: 32.1 ms per loop
100000 loops, best of 3: 13 µs per loop


In [9]:
#### iDFT Calculation ####
M2 = np.exp(2j * np.pi * k * m / n)
x2 = np.dot(y, M2) / n
print(np.allclose(x, x2))
print(np.allclose(x, np.fft.ifft(y)))

True
True


In [10]:
#### Components in numpy.fft ####
a = np.random.randint(10, size = 10)
print("a = {}, mean of a is {}".format(a, a.mean()))
A = np.fft.fft(a)
print("A[0] / length = {}".format(A[0] / 10))
print("Both Positive and Negative terms of A: {}".format(A[int(10 / 2)]))

a = [0 4 9 9 1 0 3 7 6 7], mean of a is 4.6
A[0] / length = (4.6+0j)
Both Positive and Negative terms of A: (-7.999999999999997+1.1546319456101628e-14j)


In [11]:
#### Shift Zero-frequency ####
np.fft.fftshift(A)

array([ -8.00000000 +1.15463195e-14j,  -2.55572809 -1.53884177e+00j,
        -1.35410197 -7.69420884e+00j, -20.44427191 -3.63271264e-01j,
         5.35410197 +1.81635632e+00j,  46.00000000 +0.00000000e+00j,
         5.35410197 -1.81635632e+00j, -20.44427191 +3.63271264e-01j,
        -1.35410197 +7.69420884e+00j,  -2.55572809 +1.53884177e+00j])

In [12]:
#### Two-dimensional Fourier Transform ####
x = np.random.random(24)
x.shape = 2,12
y2 = np.fft.fft2(x)
x.shape = 1,2,12
y3 = np.fft.fftn(x, axes = (1, 2))
print(np.allclose(y2, y3))

True


### Application: Interpolation

In [13]:
#### Original Image ####
from matplotlib import image
img = image.imread('./scientist.png')
gray_img = np.dot(img[:,:,:3], [.21, .72, .07])
print(gray_img.shape)
plt.imshow(gray_img, cmap = plt.get_cmap('gray'))
plt.show()

(317, 661)


In [14]:
#### Image Spectrum ####
fft = np.fft.fft2(gray_img)
amp_spectrum = np.abs(fft)
plt.imshow(np.log(amp_spectrum))
plt.show()

In [15]:
#### Replace the Zero-frequency Component to the Center ####
fft_shift = np.fft.fftshift(fft)
plt.imshow(np.log(np.abs(fft_shift)))
plt.show()

In [20]:
#### Created Interpolated/ Enlarged Image ####
m, n = fft_shift.shape
b = np.zeros((int(m / 2), n))
c = np.zeros((2 * m - 1, int(n / 2)))
fft_shift = np.concatenate((b, fft_shift, b), axis = 0)
fft_shift = np.concatenate((c, fft_shift, c), axis = 1)
ifft = np.fft.ifft2(np.fft.ifftshift(fft_shift))
ifft.shape
ifft = np.real(ifft)
plt.imshow(ifft, cmap = plt.get_cmap('gray'))
plt.show()